# Overview

This notebook outlines a workflow and executes key code for batch processing of autotracking.



# Packages, paths, and parameters
You'll need to execute the next cell for all of the code that follows. Make sure that the root_code and root_proj paths are correct for your system.

In [1]:
import sys
import os
import def_definepaths as dd
import def_acquisition as da
import numpy as np
import cv2 as cv

# Root path for code on your system (can be set to None if you want to use give_paths to define them)
root_code = None

# Root path for the data and videos for the project
root_proj = None

# Get paths (specific to system running code)
path = dd.give_paths(root_code=root_code, root_proj=root_proj)

# Add path to kineKit 'sources' directory
sys.path.insert(0, path['kinekit'] + os.sep + 'sources')

# Import from kineKit
import acqfunctions as af
import videotools as vt

# Raw video extension
vid_ext_raw = 'MOV'

# Compressed video quality level (0 to 1, where 1 is no compression)
vid_quality = 0.75

# Create mask

You'll need to create a new mask whenever the position of the tank changes in the video frame. Follow the following steps to create a new mask.

## Measure the region-of-interest (ROI)

Here the aim is to define an ellipical region-of-interest where the tank resides within the video frames.

1. Set the 'make_video' column in the 'experiment_log.csv' file to a value of 1 for the video for which you want to create a mask.

1. Set all other 'make_video' rows to 0. If there is more than one row set to 1 then the code will use the first in the list.

1. Run the cell below to save a single frame from the video, which is saved to the 'masks' directory.

1. After the frame is generated, open it up in ImageJ, use the ellipse tool to draw around the tank margin. Be sure not to cut off any portion of the tank where a fish might end up. All areas outside of the ROI will be ignored.

1. Once the ellipse has been drawn, select Analyze:Measure in the pull-down menus to find the region-of-interest coordinates.

1. Enter the value for 'BX' as 'roi_x' in the experiment_log (enter values online in Google Sheets). Do the same for BY->roi_y, Width->roi_w, Height->roi_h. Copy and paste the values to all rows corresponding to videos that use that same ROI.

1. Make sure that the local version of experiment_log matches the values as the Google Sheet. This can be done by redownloading the CSV file, or copy and pasting values to the local copy. 



In [55]:
# Index of video in cat list to extract video
vid_index = 0

# Extract experiment catalog info
cat = af.get_cat_info(path['cat'])

# Filename for frame of current sequence
filename = cat.date[vid_index] + '_' + format(cat.exp_num[vid_index],'03') + '_frame1'

# Define path for video
full_path = path['vidin'] + os.sep + cat.video_filename[vid_index] + '.' + vid_ext_raw

# Extract frame and save to 'mask' directory
im = vt.get_frame(full_path)
cv.imwrite(path['mask'] + os.sep + filename + '.jpg', im)

[ WARN:0@11117.722] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@11117.722] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin3 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@11117.722] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@11117.722] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/openc

True

## Create the mask image

1. As in step above, the cell below will use the first row for which make_video=1 in the 'experiment_log.csv' file to define the ROI for the mask, so adjust the spreadsheet accordingly.

1. The code will prompt you to choose a filename for the mask image and will save that file to the 'masks' directory.

1. Once completed, enter the mask filename (without the 'png' extension) into the mask_filename column of experiment_log for all experiments that should use that mask.

In [6]:
# Index of video in cat list to extract video
vid_index = 0

# Extract experiment catalog info
cat = af.get_cat_info(path['cat'])

# Define path to raw video
full_path = path['vidin'] + os.sep + cat.video_filename[vid_index] + '.' + vid_ext_raw

# Extract video frame 
im = vt.get_frame(full_path)

# Make frame a gray field
im = int(256/2) + 0*im

# Extract roi coordinates
x_roi = float(cat.roi_x[vid_index])
y_roi = float(cat.roi_y[vid_index])
w_roi = float(cat.roi_w[vid_index])
h_roi = float(cat.roi_h[vid_index])
xC = x_roi + w_roi/2
yC = y_roi + h_roi/2
dims = (int(np.ceil(w_roi/2)), int(np.ceil(h_roi/2)))
cntr = (int(x_roi + w_roi/2), int(y_roi + h_roi/2))

# Define transparent image for mask
im = cv.ellipse(im, cntr, dims, angle=0, startAngle=0, endAngle=360, color=(255,255,255),thickness=-1)
trans_img = int(255/3) * np.ones((im.shape[0], im.shape[1], 4), dtype=np.uint8)
trans_img[np.where(np.all(im[..., :3] == 255, -1))] = 0

# Filename for frame of current sequence
filename = cat.date[vid_index] + '_' + format(cat.exp_num[vid_index],'03') + '_mask'

# Write mask image to disk
cv.imwrite(path['mask'] + os.sep + filename + '.png', trans_img)


[ WARN:0@1292.406] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2386) handleMessage OpenCV | GStreamer warning: your GStreamer installation is missing a required plugin
[ WARN:0@1292.406] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (2402) handleMessage OpenCV | GStreamer warning: Embedded video playback halted; module uridecodebin4 reported: Your GStreamer installation is missing a plug-in.
[ WARN:0@1292.406] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (1356) open OpenCV | GStreamer warning: unable to start pipeline
[ WARN:0@1292.406] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-su

True

# Create compressed movies

The code below will generate compressed videos for all experiments in experiment_log where analyze=1 and make_video=1. 
This is done in two steps.
First, uncompressed masked movies are created and stored in the 'tmp' directory, then compressed and cropped movies are saved in 'pilot_compressed' (the tmp movies are then deleted).

To speed up this step, we've parallelized the code.
So, you will want to adjust the num_cores parameter below to the number of cores in your machine.
However, the code cannot handle a situations where you are converting more movies than there are cores.
So, if you are converting fewer movies than you have cores, then allocate the number of cores to the total number of movies.

In order for the parallel processing to work, open a terminal and activate the environment for this project (e.g., 'conda active wake' for the 'wake' environment), and run the following code, where the final number is the number of cores to be run:

> ipcluster start -n 8

You should get a message that "Engines appear to have started successfully", if things are working.

Next, execute the batch_command function below.

In [ ]:
def batch_command(cmds):
    """ Runs a series of command-line instructions (in cmds dataframe) in parallel """

    import ipyparallel as ipp
    import subprocess

    # Set up clients 
    client = ipp.Client()
    type(client), client.ids

    # Direct view allows shared data (balanced_view is the alternative)
    direct_view = client[:]

    # Function to execute the code
    def run_command(idx):
        import os
        output = os.system(cmds_run.command[idx])
        # output = subprocess.run(cmds_run.command[idx], capture_output=True)
        # result = output.stdout.decode("utf-8")
        return output
        # return idx

    direct_view["cmds_run"] = cmds

    res = []
    for n in range(len(direct_view)):
        res.append(client[n].apply(run_command, n))
    
    return res

The compressed videos are created in two steps. 
First, uncompressed videos are generated with a mask and saved in the 'tmp' folder. 
The cell below accomplishes this step, but note that the work is accomplished by sending the job to the terminal, which makes it look like here like the job is complete. 
After you have started the job, you can track its progress below.

In [ ]:
# Extract experiment catalog info
cat = af.get_cat_info(path['cat'])

# Make the masked videos (stored in 'tmp' directory)
cmds = af.convert_masked_videos(cat, in_path=path['vidin'], out_path=path['tmp'], 
            maskpath=path['mask'], vmode=False, imquality=1, para_mode=True, echo=False)

# Run FFMPEG commands in parallel
results = batch_command(cmds)


You can check on the status of this job with the print command in the cell below.
For each video that is being worked on, you should see this result:

> <AsyncResult(run_command): pending>

However, if there is a problem, then you will see something like this:

> <AsyncResult(run_command): failed>

When the job finishes correctly, then it will look like this:

> <AsyncResult(run_command): finished>

You can keep re-running this cell each time you want to check on its status. If you have a lot of movies, then this can take a long time. 
Do not move to the next step until all videos are finished.

In [ ]:
print(results)

In the second step, the masked videos are compressed and cropped with parallel processing.
This is achieved in a similar way, with the following cell.

In [ ]:
cmds = af.convert_videos(cat, in_path=path['tmp'], out_path=path['vidout'], 
            vmode=False, imquality=vid_quality, suffix_in='mp4', para_mode=True, echo=False)

# Run FFMPEG commands in parallel
results = batch_command(cmds)

Again, you can check on the job status:

In [ ]:
print(results)

Once the job is finished, then you can survey the directories to make sure that all the videos have been compressed and the tmp files will be deleted:

In [ ]:
# Re-extract experiment catalog info
cat = af.get_cat_info(path['cat'])

# Step through each experiment
for c_row in cat.index:

    # Input video path
    vid_in_path = path['vidin'] + os.sep + cat.video_filename[c_row] + '.' + os.sep + vid_ext_raw

    # Temp video path
    vid_tmp_path = path['tmp'] + os.sep + cat.video_filename[c_row] + '.mp4'

    # Output video path
    vid_out_path = path['vidout'] + os.sep + cat.video_filename[c_row] + '.mp4'

    # Check whether output file was not made
    if not os.path.isfile(vid_out_path):

        print('   Output movie NOT created successfully: ' + vid_out_path)

        if os.path.isfile(vid_tmp_path):
            print('   Also, temp. movie NOT created successfully: ' + vid_tmp_path)
        else:
            print('   But, temp. movie created successfully: ' + vid_tmp_path)

    # If it was . . .
    else:
        print('   Output movie created successfully: ' + vid_out_path)

        # Delete temp file
        if os.path.isfile(vid_tmp_path):
            os.remove(vid_tmp_path)

# TRex tracking